# 論文輪読会＃3　ResNet

## 1. 論文を読む際に注意すべき観点

- どんなもの？

2015年のImageNetコンペティションとCOCOセグメンテーションの最良モデルとして<br>
Microsoft Researchから提案され、最大1000層以上の深いニューラルネットワークを<br>
構築することが可能となった。

<img src="https://deepage.net/img/convolutional_neural_network/conv.jpg" height="150px">

<img src="https://deepage.net/img/resnet/layers_of_ilsvrc.jpg" height="150px">

- 先行研究と比べてどこがすごい？

Convolution層はフィルタを持ち、Pooling層と組み合わせて何らかの特徴を検出する役割を持っている。<br>
低・中・高レベルの特徴を多層形式で自然に統合し、認識レベルを強化することができる。

ResNet以前も層を更に深くする試みはあったものの、思い通りに学習が進まなかった。<br>
単純に層を深くすると、性能が悪化することが報告されていた。
<img src="https://deepage.net/img/resnet/not_easy_deep.jpg" height="100px">

- 技術や手法のキモはどこ？

「ある層で求める最適な出力を学習するのではなく、層の入力を参照した残差関数を学習する」 <br>
ことで最適化しやすくしている。

- どうやって有効だと検証した？

層を深くしても性能向上が見られた<br>
<img src="15_imgnet/img303.png" width="650px" height="650px">

- 議論はある？

層が深いゆえに**「学習時間が極端に長い」** 

最初のResNetの論文では8枚のGPUを稼働させながら学習に数週間を要した(152層のモデル)。<br> 
→豊富な時間と整った環境はResNetを用いる ための最低条件

- 次に読むべき論文は？

派生研究を参照。

## 2. 技術のキモ

### 2-1. 用語

- Plaint net<br>
従来型のネットワーク

- Residual net<br>
shortcut connection を持つネットワーク

### 2-2. ResNet の構成

<img src="15_imgnet/img300.png" width="300px" height="300px">

deep pathが従来のルート。 <br>
shortcut connectionは右図赤線のルート。

重みの更新もdeep pathでF(x)を出力するように学習 されます。<br>
残差(residue)を出力するNetworkであることから **ResNet(Deep Residual Network)**と呼ぶ。


### 2-3. なぜ勾配問題が解決できたのか？ 

<img src="15_imgnet/img301.png" width="380px" height="380px">

もし重みw^(l+2)->0,b^(l+2)->0 の場合、出力が層を通過する前の入力値と同じになる

gは活性化関数

残差ブロックでは、畳込み層とSkip Connectionの組み合わせになっている。<br>
2つの枝から構成されていて、それぞれの要素を足し合わせる。<br>
残差ブロックの一つはConvolution層の組み合わせで、もう一つはIdentity関数となる。

こうすれば、仮に追加の層で変換が不要でもweightを0にすれば良い。

残差ブロックを導入することで、結果的に**層の深度の限界を押し上げることができ**、<br>
精度向上を果たすことが出来た。

### 2-4. なぜ性能が高いのか？ 

Andreas(2016)らの論文では<br>
ResNetの予測精度および汎用性の高さの根拠を<br>
「並列パスのアンサンブル学習とみなせるから」<br>
と解釈している。 

・Resnetが下のような並列のパスに書き換え可能で、<br>
・各々のパスが別々の 学習器として扱えるアンサンブル学習として振る舞っている。

論文内では実験として<br>
- deep pathを1箇所取り除きshortcut connectionのみ残し たところ、<br>
  精度にはほとんど影響がなかった(パス同士に依存性がない)<br>
- 取り除 くdeep pathを増やしたところ、取り除いたdeep pathの数に応じて<br>
  徐々に精度が 低下した

様子が示されており、各々のパスが確かにアンサンブル学習的振る舞 いを<br>
見せていることが分かる。 

### 2-5. 派生研究

- 100層を超えるResNetは学習に数週間を要する。改善できないか？ <br>
→深い層ではdeep pathが0に近い値を出力することが知られている。<br> 
　Gao(2016)らの論文ではそれを利用し、**深い層において一定確率で deep pathを<br>
　無視し、shortcut connectionを素通りする**ようなモデルを作成。<br>
　精度を下げることなく25%程度の時間短縮に成功<br>
　(Deep Networks with Stochastic Depth)。 <br>
 
https://arxiv.org/abs/1603.09382<br>
https://arxiv.org/pdf/1603.09382.pdf

**The survival probabilities** (p.5-6)　　L:ResBlocks の総数、l:ResBlocksの番号<br>
$p_l$ are new hyper-parameters of our training procedure.<br>
Intuitively, they should take on similar values for neighboring ResBlocks.<br>
One option is to set $p_l = p_L$ uniformly for all $p_L$.<br>
Another possibility is to set them according to a smooth function of $l$.<br>
We propose a simple linear decay rule from $p_0 = 1$ for the input, to $p_L$<br>
for the last ResBlock:

\begin{equation*}
p_l = 1 − \frac{l}{L}(1 − p_L) . 
\end{equation*}

Training time savings. When a ResBlock is bypassed for a specific iteration,
there is no need to perform forward-backward computation or gradient updates.
As the forward-backward computation dominates the training time, stochastic
depth significantly speeds up the training process. Following the calculations
above, **approximately 25% of training time could be saved** under the linear decay
rule **with pL = 0.5**. The timings in practice using our implementation are
consistent with this analysis (see the last paragraph of Section 4). More computational
savings can be obtained by switching to a uniform probability for
$p_l$ or lowering $p_L$ accordingly. In fact, Fig. 8 shows that **with $p_L$ = 0.2**, the
ResNet with stochastic depth obtains the same test error as its constant depth
counterpart on CIFAR-10 but gives a **40% speedup**.

- 層を深くするのでなく、畳み込み時のフィルタの数を増やして <br>
  ネットワークを広くする事で性能を上げられないか？<br>
→Sergeyらの論文によると、**ネットワークを広げる**事によって<br>
　  僅か**16層程度のモデル**でも既存のResNetの性能を出す事が可能に。 <br>
  　層数も少なくできるので**学習時間の削減**にも成功(Wide ResNet)。
   
 https://arxiv.org/abs/1605.07146<br>
 https://arxiv.org/pdf/1605.07146.pdf

<img src="15_imgnet/img306.png" width="300px" height="300px">

We show that our best CIFAR wide WRN-28-10 is **1.6 times faster** than thin ResNet-1001. <br>
Furthermore, wide WRN-40-4, which has approximately the same accuracy as ResNet-1001, is **8 times faster**.

- Deep path内の**バッチ正規化や畳み込み層の順序**の考察。<br>
→左・青枠が元論文のモデル。右・赤枠がKaimingらの改良ネットワーク<br>
　改良モデルの方が性能が高い。 合流後のReLUは不要。<br>

<img src="15_imgnet/img302.png" width="300px" height="300px">

一般にResidual Networksというと現在はこれを指します。 <br>
http://kaiminghe.com/ilsvrc15/ilsvrc2015_deep_residual_learning_kaiminghe.pdf

**bottleneck**<br>
<img src="15_imgnet/img305.png" width="450px" height="450px">

## 3. 論文精読


### Abstract

Deeper neural networks are more difficult to train. We present a residual learning framework to ease the training of networks that are substantially deeper than those used previously. 
We explicitly reformulate the layers as learning residual functions with reference to the layer inputs, instead of learning unreferenced functions. 

これまでに使用されていたものよりもかなり深いネットワークのトレーニングを容易にするために、<br>
残差学習フレームワークを提示します。非参照関数を学習するのではなく、レイヤ入力を参照して<br>
残差関数を学習するようにレイヤを明示的に再構成する。

We provide comprehensive empirical evidence showing that these residual networks are easier to optimize, and can gain accuracy from considerably increased depth. On the ImageNet dataset we evaluate residual nets with a depth of up to 152 layers—8×deeper than VGG nets [41] but still having lower complexity. 
An ensemble of these residual nets achieves 3.57% error on the ImageNet test set. This result won the 1st place on the ILSVRC 2015 classification task. We also present analysis on CIFAR-10 with 100 and 1000 layers.

残差ネットワークがより容易に最適化され、かなり深い深度生成から精度を得ることができる<br>
という包括的な経験的証拠を提供する。

ImageNetデータセットでは、VGGネットよりも深い8×15の深さの残差ネットを評価しますが、<br>
それでも複雑さは低くなります。

これらの残差ネットのアンサンブルは、ImageNetテストセットで3.57％のエラーを達成します。<br>
この結果は、ILSVRC 2015分類作業の第1位を獲得しました。<br>
我々はまた、100および1000層のCIFAR-10に関する分析を提示する。

The depth of representations is of central importance for many visual recognition tasks. 
Solely due to our extremely deep representations, we obtain a 28% relative improvement on the COCO object detection dataset. 
Deep residual nets are foundations of our submissions to ILSVRC & COCO 2015 competitions1, where we also won the 1st places on the tasks of ImageNet detection, ImageNet localization, COCO detection, and COCO segmentation.

表現の深さは、多くの視覚的認識課題にとって重要である。<br>
私たちの非常に深い表現のために、私たちはCOCOオブジェクト検出データセットで<br>
28％の相対的な改善を得ています。

深い残留網は、ILSVRC＆COCO 2015の競技会1への提出の基礎であり、ImageNet検出、<br>
ImageNetローカリゼーション、COCO検出、およびCOCOセグメンテーションの第1位を獲得しました。


### 3-1. Introduction

Deep convolutional neural networks [22, 21] have led to a series of breakthroughs for image classification [21, 50, 40]. Deep networks naturally integrate low/mid/high-level features [50] and classifiers in an end-to-end multi-layer fashion, and the “levels” of features can be enriched by the number of stacked layers (depth). Recent evidence [41, 44] reveals that network depth is of crucial importance, and the leading results [41, 44, 13, 16] on the challenging ImageNet dataset [36] all exploit “very deep” [41] models, with a depth of sixteen [41] to thirty [16]. Many other nontrivial visual recognition tasks [8, 12, 7, 32, 27] have also greatly benefited from very deep models.

深い畳み込みニューラルネットワーク[22、21]は、画像分類のための一連のブレークスルーを導いた[21,50,40]。

ディープネットワークは、**低/中/高レベルの特徴[50]と分類器**を<br>
**エンドツーエンドの多層式で自然に統合**し、**特徴の「レベル」を積み重ねられた層の数（深さ）<br>
で豊かに**することができる。

最近の証拠[41,44]は、ネットワークの深さが非常に重要であることを示している<br>


Driven by the significance of depth, a question arises: Is learning better networks <br>
as easy as stacking more layers? 

An obstacle to answering this question was the notorious problem of vanishing/exploding gradients [1, 9], <br>
which hamper convergence from the beginning. 

This problem, however, has been largely addressed by normalized initialization [23, 9, 37, 13] <br>
and intermediate normalization layers [16], which enable networks with tens of layers <br>
to start converging for stochastic gradient descent (SGD) with backpropagation [22].

より多くのレイヤーを積み重ねることは簡単ではなく、
勾配の消滅/爆発勾配問題があった。[1,9]。

<img src="https://arxiv-sanity-sanity-production.s3.amazonaws.com/render-output/64285/x1.png" width="650px" height="650px">

#### 深いネットワークの精度劣化問題

When deeper networks are able to start converging, a degradation problem has been exposed: with the network depth increasing, accuracy gets saturated (which might be unsurprising) and then degrades rapidly. Unexpectedly, such degradation is not caused by overfitting, and adding more layers to a suitably deep model leads to higher training error, as reported in [11, 42] and thoroughly verified by our experiments. Fig. 1 shows a typical example.

より深いネットワークが収束を開始できるようになると、劣化の問題が明らかになりました。<br>
ネットワークの深さが深くなると、精度は飽和してしまいます（これは驚異的かもしれません）。

予期しないことに、このような劣化は過補正によって引き起こされるものではなく、<br>
適切な深いモデルに層を追加すると、[11、42]で報告されているように、<br>
トレーニング誤差が大きくなり、図1に代表例を示す。

#### システムの最適化は容易ではない

**The degradation (of training accuracy) indicates that not all systems are similarly easy to optimize**. 

Let us consider a shallower architecture and its deeper counterpart that adds more layers onto it. There exists a solution by construction to the deeper model: the added layers are identity mapping, and the other layers are copied from the learned shallower model. The existence of this constructed solution indicates that a deeper model should produce no higher training error than its shallower counterpart. But experiments show that our current solvers on hand are unable to find solutions that are comparably good or better than the constructed solution (or unable to do so in feasible time).


学習効率の低下は、すべてのシステムが同様に容易に最適化できるわけではないことを示しています。

より浅いアーキテクチャと、より多くのレイヤを追加する深い対応を考えてみましょう。
より深いモデルへの構築による解決策が存在する。

追加された層はアイデンティティマッピングであり、他の層は学習された浅いモデルからコピーされる。
この構築された解の存在は、より深いモデルが、より浅いモデルよりも高い訓練エラーを生成してはならないことを示す。

しかし、実験では、現在のソルバーが、構築されたソリューションと同等かそれ以上の優れたソリューションを
見つけることができないことを示しています（実現可能な時間にそうすることはできません）。

####  この論文のテーマはネットワークが深くなったときの劣化を防ぐこと

In this paper, we address the degradation problem by introducing a deep residual learning framework. Instead of hoping each few stacked layers directly fit a desired underlying mapping, we explicitly let these layers fit a residual mapping. Formally, denoting the desired underlying mapping as $H(x)$, we let the stacked nonlinear layers fit another mapping of $F(x):=H(x)−x$.

本稿では、深い残存学習フレームワークを導入して劣化問題に取り組む。
少数の積み重ねられた各層が所望の基礎となるマッピングに直接適合することを期待する代わりに、
これらの層を残差マッピングに明示的に適合させる。

The original mapping is recast into F(x)+x. 
We hypothesize that it is easier to optimize the residual mapping than to optimize the original, unreferenced mapping. To the extreme, if an identity mapping were optimal, it would be easier to push the residual to zero than to fit an identity mapping by a stack of nonlinear layers.

元の写像はF（x）+ xに再構築されます。
元の参照されていないマッピングを最適化するよりも残差マッピングを最適化する方が簡単であると仮定します。

極端な場合、アイデンティティマッピングが最適であれば、非線形レイヤのスタックによる
アイデンティティマッピングを適合させるよりも、残差をゼロにプッシュする方が簡単です。

#### shortcut connection はF(x)+x で実現する

The formulation of F(x)+x <br>
can be realized by feedforward neural networks with “**shortcut connections**” (Fig. 2). 

Shortcut connections [2, 34, 49] are those skipping one or more layers. In our case, <br>
the shortcut connections simply perform identity mapping, and their outputs are added <br>
to the outputs of the stacked layers (Fig. 2). 

Identity shortcut connections add neither extra parameter nor computational complexity. 

The entire network can still be trained end-to-end by SGD with backpropagation, and can be easily implemented using common libraries (\eg, Caffe [19]) without modifying the solvers.

#### Residual Networkの効果

We present comprehensive experiments on ImageNet [36] to show the degradation problem and evaluate our method. We show that: 1) Our extremely deep residual nets are easy to optimize, but the counterpart “plain” nets (that simply stack layers) exhibit higher training error when the depth increases; 2) Our deep residual nets can easily enjoy accuracy gains from greatly increased depth, producing results substantially better than previous networks.


我々は、劣化問題を示すためにImageNet [36]に関する包括的な実験を行い、我々の方法を評価する。 

1）非常に深い残存ネットは最適化が容易ですが、相手の「plain」ネット（単純にスタックレイヤー）は<br>
**深度が増すと訓練エラーが高く**なります。 

2）私たちの深い残差ネットは、深度が大幅に増えることで容易に精度を向上させることができ、<br>
結果は**従来のネットワークよりも大幅に向上**します。

Similar phenomena are also shown on the CIFAR-10 set [20], suggesting that the optimization difficulties and the effects of our method are not just akin to a particular dataset. We present successfully trained models on this dataset with over 100 layers, and explore models with over 1000 layers.

同様の現象がCIFAR-10セット[20]にも示されており、最適化の困難さと本発明の方法の効果が
特定のデータセットに似ているわけではないことを示唆している。

このデータセットでは、100を超えるレイヤーで訓練されたモデルを正常に提示し、
1000以上のレイヤーを持つモデルを探索します。

On the ImageNet classification dataset [36], we obtain excellent results by extremely deep residual nets. Our 152-layer residual net is the deepest network ever presented on ImageNet, while still having lower complexity than VGG nets [41]. Our ensemble has 3.57% top-5 error on the ImageNet test set, and won the 1st place in the ILSVRC 2015 classification competition. The extremely deep representations also have excellent generalization performance on other recognition tasks, and lead us to further win the 1st places on: ImageNet detection, ImageNet localization, COCO detection, and COCO segmentation in ILSVRC & COCO 2015 competitions. This strong evidence shows that the residual learning principle is generic, and we expect that it is applicable in other vision and non-vision problems.

ImageNet分類データ[36]では、非常に深い残差ネットによる優れた結果が得られます。

私たちの152層の残存ネットは、ImageNet上でこれまでに発表された最も深いネットワークですが、
**VGGネットよりも複雑さは低い**です[41]。

私たちのアンサンブルはImageNetテストセットで**3.57％のトップ5エラー**を抱え、<br>
ILSVRC 2015分類競技会で1位を獲得しました。

また、ILSVRC＆COCO 2015の競技では、ImageNet検出、ImageNetローカリゼーション、COCO検出、<br>
COCOセグメンテーションなど、第1位を獲得しました。<br>
この強力な証拠は、残存学習原理が一般的であることを示しており、他の視覚および非視覚の問題に<br>
適用可能であると我々は期待している。

### 3-2. Related Work

**Residual Representations**

In image recognition, VLAD [18] is a representation that encodes by the residual vectors with respect to a dictionary, and Fisher Vector [30] can be formulated as a probabilistic version [18] of VLAD. Both of them are powerful shallow representations for image retrieval and classification [4, 48]. 

For vector quantization, encoding residual vectors [17] is shown to be more effective than encoding original vectors.

画像認識では、VLAD [18]は辞書に対して残差ベクトルで符号化する表現<br>
Fisher Vector [30]はVLADの確率的バージョン[18]として定式化することができる。

両方とも、画像検索と分類のための強力な浅い表現である[4、48]。

ベクトル量子化では、符号化残差ベクトル[17]は、元のベクトルを符号化するよりも<br>
効果的であることが示されている。

http://www.isi.imi.i.u-tokyo.ac.jp/~harada/pdf/IBIS_harada20111111.pdf

**VLAD(Vector of Locally Aggregated Descriptors)**
<img src="15_imgnet/img304.png" width="４00px" height="４00px">

**フィッシャーカーネル（Fisher Kernel)**

生成的アプローチ（generative approach）と識別的アプローチ（discriminative approach）を<br>
結合させる強力な枠組み→**確率分布の空間に適切な距離計量を埋め込む**

- 確率分布のなす空間は，Fisher 情報行列を計量とするリーマン空間
- 手順
 1. 局所特徴を生成する確率密度分布から導出される勾配ベクトルの計算
 2. 画像を表現する一つの特徴ベクトルの計算 →フィッシャーベクトル（Fisher Vector）
 3. 得られた特徴ベクトルを識別的分類機に入力する．


In low-level vision and computer graphics, for solving Partial Differential Equations (PDEs), the widely used Multigrid method [3] reformulates the system as subproblems at multiple scales, where each subproblem is responsible for the residual solution between a coarser and a finer scale. 

An alternative to Multigrid is hierarchical basis preconditioning [45, 46], which relies on variables that represent residual vectors between two scales. 

It has been shown [3, 45, 46] that these solvers converge much faster than standard solvers that are unaware of the residual nature of the solutions. 
These methods suggest that a good reformulation or preconditioning can simplify the optimization.

部分微分方程式（PDEs）を解くための低レベルのビジョンとコンピュータグラフィックスでは、広く使用されている<br>
Multigrid法[3]は、各サブ問題が粗いものと細かいものの間の残余解を担う複数のスケールでサブ問題としてシステムを再定式化する規模。 

Multigridの代わりに、階層ベースの事前調整[45、46]があります。

これは、2つのスケール間の残差ベクトルを表す変数に依存しています。<br>
これらのソルバーは、ソリューションの残りの性質を認識していない標準ソルバーよりも<br>
はるかに速く収束することが[3、45、46]示されています。

これらの方法は、良好な再調整またはプレコンディショニングが最適化を単純化できることを示唆している。

**Shortcut Connections**

Practices and theories that lead to shortcut connections [2, 34, 49] have been studied for a long time. An early practice of training multi-layer perceptrons (MLPs) is to add a linear layer connected from the network input to the output [34, 49]. In [44, 24], a few intermediate layers are directly connected to auxiliary classifiers for addressing vanishing/exploding gradients. The papers of [39, 38, 31, 47] propose methods for centering layer responses, gradients, and propagated errors, implemented by shortcut connections. In [44], an “inception” layer is composed of a shortcut branch and a few deeper branches.

Concurrent with our work, “highway networks” [42, 43] present shortcut connections with gating functions [15]. These gates are data-dependent and have parameters, in contrast to our identity shortcuts that are parameter-free. When a gated shortcut is “closed” (approaching zero), the layers in highway networks represent non-residual functions. On the contrary, our formulation always learns residual functions; our identity shortcuts are never closed, and all information is always passed through, with additional residual functions to be learned. In addition, highway networks have not demonstrated accuracy gains with extremely increased depth (\eg, over 100 layers).


ショートカット接続につながるプラクティスと理論[2,34,49]は長い間研究されてきた。

多層パーセプトロン（MLP）の訓練の初期の実践は、
ネットワーク入力から出力に接続された線形層を追加することである[34,49]。 

[44,24]では、いくつかの中間層が、消失/爆発勾配に対処するための補助分類器に
直接接続されている。 

[39,38,31,47]の論文は、ショートカット接続によって実装されたレイヤレスポンス、グラディエント、
伝搬エラーをセンタリングする方法を提案しています。

[44]において、「開始」層は、ショートカットブランチとより深いブランチで構成されています。

我々の研究と並行して、「高速道路ネットワーク」[42,43]は、ゲーティング機能とのショートカット接続を提示している[15]。

これらのゲートは、パラメータに依存しないIDショートカットとは異なり、データに依存し、パラメータを持っています。

ゲートされたショートカットが「クローズ」（ゼロに近づく）すると、ハイウェイネットワークのレイヤーは
非残余関数を表します。

逆に、我々の定式化は常に残余関数を学習する。アイデンティティのショートカットは決して閉じられず、<br>
すべての情報が常に通過し、追加の残余関数が学習されます。

さらに、高速道路ネットワークは、深度が非常に深く（例えば、100以上の層）、精度の向上を示さなかった。

### 3-3. Deep Residual Learning

#### Residual Learning

Let us consider 
H
(
x
)
 as an underlying mapping to be fit by a few stacked layers (not necessarily the entire net), with 
x
 denoting the inputs to the first of these layers. If one hypothesizes that multiple nonlinear layers can asymptotically approximate complicated functions2, then it is equivalent to hypothesize that they can asymptotically approximate the residual functions, \ie, 
H
(
x
)
−
x
 (assuming that the input and output are of the same dimensions). So rather than expect stacked layers to approximate 
H
(
x
)
, we explicitly let these layers approximate a residual function 
F
(
x
)
:=
H
(
x
)
−
x
. The original function thus becomes 
F
(
x
)
+
x
. Although both forms should be able to asymptotically approximate the desired functions (as hypothesized), the ease of learning might be different.




残りの学習H（x）をいくつかの積み重ねられたレイヤー（必ずしもネット全体ではない）
にフィットさせる基礎となるマッピングとして考えてみましょう。

xはこれらのレイヤーの最初のものへの入力を示します。

複数の非線形層が複雑な関数2を漸近的に近似できると仮定すると、

残差関数H（x）-x（入力と出力が同じ次元であると仮定して）を漸近的に近似できる<br>
と仮定することと等価である。

したがって、積み重ねられた層がH（x）を近似することを期待するのではなく、<br>
これらの層を残差関数F（x）：= H（x）-xに近似させる。

従って元の関数はF（x）+ xとなる。両方の形式は、漸近的に（仮定されているように）<br>
所望の関数を近似することができなければならないが、学習の容易さは異なる可能性がある。

This reformulation is motivated by the counterintuitive phenomena about the degradation problem (Fig. 1, left). As we discussed in the introduction, if the added layers can be constructed as identity mappings, a deeper model should have training error no greater than its shallower counterpart. The degradation problem suggests that the solvers might have difficulties in approximating identity mappings by multiple nonlinear layers. With the residual learning reformulation, if identity mappings are optimal, the solvers may simply drive the weights of the multiple nonlinear layers toward zero to approach identity mappings.


追加されたレイヤーをアイデンティティマッピングとして構築することができれば、より深いモデルのほうが<br>
浅いモデルよりトレーニングエラーが大きくならない


In real cases, it is unlikely that identity mappings are optimal, but our reformulation may help to precondition the problem. If the optimal function is closer to an identity mapping than to a zero mapping, it should be easier for the solver to find the perturbations with reference to an identity mapping, than to learn the function as a new one. We show by experiments (Fig. 7) that the learned residual functions in general have small responses, suggesting that identity mappings provide reasonable preconditioning.

実際には、アイデンティティマッピングが最適であるとは考えにくいが、我々の再定式化は問題を前提条件とするのに役立つかもしれない。最適な関数がゼロ写像よりもアイデンティティ写像に近い場合、ソルバは新しい写像としての写像よりもアイデンティティ写像を参照して摂動を見つけやすくなります。

学習された残余関数は一般的に小さな応答しか示さず、アイデンティティマッピングが合理的な前処理を提供することを示唆する実験（図7）を示す。

****Identity Mapping by Shortcuts****

We adopt residual learning to every few stacked layers. A building block is shown in Fig. 2. Formally, in this paper we consider a building block defined as:

$
y=F(x,{W_i})+x. (1)
$

Here $x$ and $y$ are the input and output vectors of the layers considered. 

The function $ F(x,W_i) $ represents the residual mapping to be learned. <br>
For the example in Fig. 2 that has two layers, 
$F=W_2\sigma (W_1x)$ in which $\sigma$ denotes ReLU [29] and the biases are omitted<br>
for simplifying notations. 

The operation 
$F + x$ is performed by a **shortcut connection and element-wise addition**. 

We adopt the second nonlinearity after the addition (\ie, $\sigma(y)$, see Fig. 2).

The shortcut connections in Eqn.(1) introduce neither extra parameter nor computation complexity. 

hortcut connections はパラメータを増やしもしないし、計算の複雑さも増さない。

This is not only attractive in practice but also important in our comparisons between **plain and residual networks**. We can fairly compare plain/residual networks that simultaneously have the same number of parameters, depth, width, and computational cost (except for the negligible element-wise addition).

これは、実際には魅力的であるだけでなく、単純ネットワークと残余ネットワークの比較においても重要です。

プレーン/残差ネットワークを<br>
同じ数のパラメータ、深さ、幅、および計算コストを同条件に公平に比較できる

The dimensions of $x$ and $F$ must be equal in Eqn.(1). <br>
If this is not the case (\eg, when changing the input/output channels),<br>
we can perform a linear projection $W_s$ by the shortcut connections to <br>
match the dimensions:

$y=F(x,W_i)+W_s x$. (2)

We can also use a square matrix $W_s$ in Eqn.(1). <br>
But we will show by experiments that the identity mapping is sufficient <br>
for addressing the degradation problem and is economical,<br>
and thus $W_s$ is only used when matching dimensions.<br>

The form of the residual function $F$ is flexible. <br>
Experiments in this paper involve a function $F$ that has two or three layers (Fig. 5),
while more layers are possible. 

But if $F$ has only a single layer, Eqn.(1) is similar to a linear layer: <br>
$y=W_1 + x$, for which we have not observed advantages.

We also note that although the above notations are about fully-connected layers for simplicity, 
they are applicable to convolutional layers. 

The function $F(x,{W_i})$ can represent multiple convolutional layers.<br>
The element-wise addition is performed on two feature maps, channel by channel.

<img src="https://arxiv-sanity-sanity-production.s3.amazonaws.com/render-output/64285/x3.png" width="４00px" height="４00px">


### 3-4. Network Architectures

We have tested various plain/residual nets, and have observed consistent phenomena. To provide instances for discussion, we describe two models for ImageNet as follows.



#### Plain Network. 

Our plain baselines (Fig. 3, middle) are mainly inspired by the philosophy of VGG nets [41] (Fig. 3, left). <br>
The convolutional layers mostly have 3×3 filters and follow two simple design rules:

(i) for **the same output feature map size**, the layers have **the same number of filters**; and <br>
(ii) if the feature map size is halved, the number of filters is doubled so as to preserve the time complexity per layer. 

特徴マップのサイズが半分になると、フィルタの数が2倍になり、レイヤごとに時間の複雑さが保たれます。

We perform downsampling directly by convolutional layers that have a stride of 2. The network ends with a global average pooling layer and a 1000-way fully-connected layer with softmax. The total number of weighted layers is 34 in Fig. 3 (middle).

ストライドが2の畳み込みレイヤーによって直接ダウンサンプリングを実行します。<br>
ネットワークは、グローバル平均プールレイヤーとsoftmaxを持つ1000ウェイの完全接続レイヤーで終了します。<br>
重み付けされた層の総数は図3の34

It is worth noticing that our model has fewer filters and lower complexity than VGG nets [41] (Fig. 3, left). Our 34-layer baseline has 3.6 billion FLOPs (multiply-adds), which is only 18% of VGG-19 (19.6 billion FLOPs).

VGGよりネットワークの複雑さは低い、フィルター数が少ない。
VGG-19の１８％程度

#### Residual Network. 

Based on the above plain network, we insert shortcut connections (Fig. 3, right) 
which turn the network into its counterpart residual version. 

The identity shortcuts (Eqn.(1)) can be directly used when the input and output are of 
the same dimensions (solid line shortcuts in Fig. 3). 

When the dimensions increase (dotted line shortcuts in Fig. 3), we consider two options: 

(A) The shortcut still performs identity mapping, with extra zero entries padded
for increasing dimensions. This option introduces no extra parameter; 

ショートカットは依然としてアイデンティティマッピングを実行し、次元が増えるにつれて余分な<br>
ゼロエントリが埋められます。このオプションは余分なパラメータを導入しません。

(B) The projection shortcut in Eqn.(2) is used to match dimensions 
(done by 1×1 convolutions). 

For both options, when the shortcuts go across feature maps of two sizes, 
they are performed with a stride of 2.

### Implementation

Our implementation for ImageNet follows the practice in [21, 41]. The image is resized with its shorter side randomly sampled in 
[
256
,
480
]
 for scale augmentation [41]. 
 
#### (1) Scale Augmentation の採用

拡大縮小のために、画像の短辺を[256、480]で無作為にサンプリングしてサイズ変更します[41]。 

#### (2) Per-pixel Mean Subtraction の採用

A 224×224 crop is randomly sampled from an image or its horizontal flip, with the per-pixel mean subtracted [21].

224×224のクロップは、画像またはその水平フリップから無作為にサンプリングされ、
「Per-pixel Mean Subtraction」を実施。
 
224x224x3 個の値について個別に平均を計算し、平均を引く。

#### (3) Color Augmentation の採用

The standard color augmentation in [21] is used. 

画像の明るさを調整、画像のコントラストを調整

各ピクセルの RGB を 3 次元のベクトルの集合だと考え PCA をかける。<br>
ガウス分布でノイズを生成し、固有ベクトル方向にノイズを加える。<br>
乱数は各ピクセルではなくパッチ全体に対して共通。

#### (4) batch normalization の採用

We adopt batch normalization (BN) [16] right **after each convolution and before activation**,
following [16]. 

各畳み込みの直後、活性化関数の前に、バッチ標準化（BN）[16]を採用。

#### (5) 初期化、ミニバッチ、学習率、weight decay、momentum

We initialize the weights as in [13] and train all plain/residual nets from scratch. We use SGD with a mini-batch size of 256. The learning rate starts from 0.1 and is divided by 10 when the error plateaus, and the models are trained for up to 
60
×
10
4
 iterations. <br>
 We use a weight decay of 0.0001 and a momentum of 0.9.
 
- [13]のように重みを初期化し、全てのプレーン/残差ネットをゼロから訓練する。
- 最小バッチサイズ256のSGDを使用。
- 学習率は0.1から始まり、エラーが起きたときに10で除算され、<br>
- モデルは60×10 4回の反復までトレーニングされます。 
- 0.0001の重量減と0.9の運動量を使用します。

#### (6) ドロップアウトは利用しない

We do not use dropout [14], following the practice in [16].



#### (7) 10-crop Testing の採用

In testing, for comparison studies we adopt the standard 10-crop testing [21]. 

For best results, we adopt the fully-convolutional form as in [41, 13], and average the scores at multiple scales (images are resized such that the shorter side is in 
{
224
,
256
,
384
,
480
,
640
}
).

10 個のパッチを切り出し、それぞれに対する予測を平均して答える。

最良の結果を得るには、[41、13]のように完全畳み込み形式を採用し、<br>
複数のスケールで平均をとる（画像のサイズを変更して、短辺が{224,256,384,480,640}になるようにする）。

## Experiments /  ImageNet Classification

#### Deeper Bottleneck Architectures. 

Next we describe our deeper nets for ImageNet. Because of concerns on the training time that we can afford, we modify the building block as a bottleneck design4. For each residual function 
F
, we use a stack of 3 layers instead of 2 (Fig. 5). The three layers are 1
×
1, 3
×
3, and 1
×
1 convolutions, where the 1
×
1 layers are responsible for reducing and then increasing (restoring) dimensions, leaving the 3
×
3 layer a bottleneck with smaller input/output dimensions. Fig. 5 shows an example, where both designs have similar time complexity.

トレーニング時間に懸念があるため、ビルディングブロックをボトルネックの設計として修正します。

各残余関数Fについて、2の代わりに3つの層のスタックを使用する（図5）。 3つの層は、1×1、3×3、および1×1の畳み込みであり、1×1層は寸法を縮小してから増加させ（復元する）、3×3層は入力/出力の寸法が小さい。図5は、両方の設計が同様の時間の複雑さを有する例を示す。

The parameter-free identity shortcuts are particularly important for the bottleneck architectures. If the identity shortcut in Fig. 5 (right) is replaced with projection, one can show that the time complexity and model size are doubled, as the shortcut is connected to the two high-dimensional ends. So identity shortcuts lead to more efficient models for the bottleneck designs.

パラメータのないアイデンティティ・ショートカットは、ボトルネック・アーキテクチャにとって特に重要です。

図5（右）のアイデンティティ・ショートカットを投影に置き換えると、ショートカットが2つの高次元の端に接続されるため、時間の複雑さとモデルのサイズが2倍になっていることがわかります。したがって、アイデンティティのショートカットは、ボトルネックの設計のためのより効率的なモデルにつながります。

## 参考：pointwise convolution(1x1の畳み込み)

<img src="./15_imgnet/img204.png"
  width="350px">

上記畳み込みフィルタを必要な出力層（Lafter）分だけ適応して、重ねることで<br>
**任意の総数の出力を得る**ことができる。<br>
この図では、層数を増やす形で畳み込みフィルタを適応している。

最大のメリットは層数を簡単に任意の値に変更できる点。

- 1x1の畳み込みはCPUでも高速で動作する（CPUとGPUの速度差が比較的小さい）
- 畳み込みのフィルタサイズを1x1にするだけで実装できるため他の次元削減・拡張手法に比べて圧倒的に実装が楽
- **出力の層数を簡単に増減できる** -> チャネル数を調整して**次元圧縮**ができる<br>
  上記の例では　192→32 へ圧縮している
- 出力にrelu等を与えることによって、**非線形性を増す**ことができる

<img src="./15_imgnet/img205.png"
  width="350px">
 
 1 × 1 convolution: 28×28×64(channel) <br>
   28×28×192 -> 28×28×64
   
 3 × 3 convolution: 28×28×128(channel) same convolution(width,heightの次元維持)<br>
   28×28×192 -> 28×28×128
   
 5 × 5 convolution: 28×28×32(channel) same convolution(width,heightの次元維持)<br>
   28×28×192 -> 28×28×32
 
 max pool:28×28×32(channel) same s=1<br>
   28×28×192 -> 28×28×32
 
 ->出力は 28×28×(32+32+128+64)=28×28×256
 
#### same/valud convolution

valid パディングを行いません<br>
same 元の入力と同じ長さを出力がもつように入力にパディングする

## 参考：計算コストとbottleneck

<img src="./15_imgnet/img206.png"
  width="350px">
  
28×28×１９２ -> 28×28×32 へ圧縮する場合(1×1 convolution未使用)<br>
   計算量は　２８×２８×１９２　×　５×５×３２＝１２０M
 
 <img src="./15_imgnet/img207.png"
  width="350px">

28×28×１９２ ->28×28×16-> 28×28×32 へ圧縮する場合(1×1 convolution使用)<br>
   計算量は　２８×２８×１９２×16 + 　２８×２８×32 × 5×5×16＝2.4 M+10.0 M=１２.4 M<< 120 M
   
**1×1 convolution使用することで、計算量は約1/10に削減されている。**

1×1 convolution の層はサイズを削減しているので、**bottleneck**と言われる。

## 参考文献

[1]ResNet の論文<br>
K. He, X. Zhang, S. Ren, J. Sun.<br>
Deep Residual Learning for Image Recognition. arXiv:1512.03385, 2015.<br>
https://arxiv.org/pdf/1512.03385.pdf

https://www.arxiv-vanity.com/papers/1512.03385/

[2]Veit, Andreas, et al.<br>
Residual Networks Behave Like Ensembles of Relatively Shallow Networks.<br>
arXiv preprint arXiv:1605.006431 (2016)

[3]Huang, Gao, et al.<br>
Deep networks with stochastic depth<br>
arXiv preprint arXiv:1603.09382 (2016) <br>

[4]　Zagoruyko & Komodakis<br>
Wide Residual Networks.<br>
arXiv preprint arXiv:1605.07146 (2016) <br>

[5] He, Kaiming, et al.<br>
Identity mappings in deep residual networks.<br>
arXiv preprint arXiv:1603.05027 (2016) 17<br>


[6]ResNetの仕組み<br>
https://www.slideshare.net/KotaNagasato/resnet-82940994

[7]Res netと派生研究の紹介<br>
https://www.slideshare.net/masatakanishimori/res-net

[8]Deep Residual Learning<br>
MSRA @ ILSVRC & COCO 2015 competitions<br>
Kaiming He<br>
with Xiangyu Zhang, Shaoqing Ren, Jifeng Dai, & Jian Sun<br>
Microsoft Research Asia (MSRA)<br>
http://kaiminghe.com/ilsvrc15/ilsvrc2015_deep_residual_learning_kaiminghe.pdf<br>

[9]KerasでVGG16を使う<br>
http://aidiary.hatenablog.com/entry/20170104/1483535144

[10]Residual Network(ResNet)の理解とチューニングのベストプラクティス<br>
https://deepage.net/deep_learning/2016/11/30/resnet.html

[11]リーマン空間の定義<br>
http://ziguzaku.hatenablog.com/entry/2014/01/21/102254

[12]Batch Normalization の理解<br>
https://qiita.com/t-tkd3a/items/14950dbf55f7a3095600